In [1]:
from simple_bank_korea.kb import get_transactions
import csv
import time
from ast import literal_eval
import paho.mqtt.client as mqtt
import pandas as pd
from sklearn.preprocessing import LabelEncoder

# 데이터 로딩, 및 특수문자 정자

In [2]:
train = pd.read_csv("C:\\Users\\munan\\Desktop\\Fam_classification\\kookmin-out.csv",encoding = 'CP949')

In [3]:
train = train[:-1]
train = train.drop('구분',1)
train = train.drop('송금메모',1)

In [4]:
k =['적요','보낸분/받는분','거래점']
for i in k:
    result=[]
    for j in range(len(train)):
        result.append(str(train[i][j]).replace('?',''))
    train[i] = result

In [5]:
train

,거래일시,적요,보낸분/받는분,출금액,입금액,잔액,거래점
0,2018.12.31 15:12:10,체크카드,루틴,"18,900",0,"259,432",KB카드
1,2018.12.29 18:25:34,체크카드,충북대학교병원,"263,730",0,"278,332",KB카드
2,2018.12.29 13:32:57,체크카드,(주)케이알산업입장(,"3,000",0,"542,062",KB카드
3,2018.12.29 13:31:16,체크카드,(주)케이알산업입장(,"1,500",0,"545,062",KB카드
4,2018.12.29 13:28:11,체크카드,(주)케이알산업입장(,"1,900",0,"546,562",KB카드
...,...,...,...,...,...,...,...
923,2018.01.04 18:04:11,국민카드,KB카드출금,"31,623",0,909,개인고
924,2018.01.03 22:58:55,직불출금,마스타해외승인출금,"2,386",0,"30,532",KB카드
925,2018.01.03 19:26:18,체크카드,(주)코리아세븐청주사,850,0,"2,918",KB카드
926,2018.01.03 19:11:06,체크카드,까구샵,"12,000",0,"3,768",KB카드


# 날짜 변수 추가 

In [6]:
train['거래일시'] = pd.to_datetime(train['거래일시'])

In [7]:
train['year'] = train['거래일시'].dt.year
train['month'] = train['거래일시'].dt.month
train['day'] = train['거래일시'].dt.day
train

,거래일시,적요,보낸분/받는분,출금액,입금액,잔액,거래점,year,month,day
0,2018-12-31 15:12:10,체크카드,루틴,"18,900",0,"259,432",KB카드,2018,12,31
1,2018-12-29 18:25:34,체크카드,충북대학교병원,"263,730",0,"278,332",KB카드,2018,12,29
2,2018-12-29 13:32:57,체크카드,(주)케이알산업입장(,"3,000",0,"542,062",KB카드,2018,12,29
3,2018-12-29 13:31:16,체크카드,(주)케이알산업입장(,"1,500",0,"545,062",KB카드,2018,12,29
4,2018-12-29 13:28:11,체크카드,(주)케이알산업입장(,"1,900",0,"546,562",KB카드,2018,12,29
...,...,...,...,...,...,...,...,...,...,...
923,2018-01-04 18:04:11,국민카드,KB카드출금,"31,623",0,909,개인고,2018,1,4
924,2018-01-03 22:58:55,직불출금,마스타해외승인출금,"2,386",0,"30,532",KB카드,2018,1,3
925,2018-01-03 19:26:18,체크카드,(주)코리아세븐청주사,850,0,"2,918",KB카드,2018,1,3
926,2018-01-03 19:11:06,체크카드,까구샵,"12,000",0,"3,768",KB카드,2018,1,3


# 보낸분/받는분, 거래점 카테고리화

In [116]:
train2 = train.copy()

In [117]:
encoder = LabelEncoder()
train2['적요'] = encoder.fit_transform(train2['적요'])

train2['보낸분/받는분'] = encoder.fit_transform(train2['보낸분/받는분'])

train2['거래점'] = encoder.fit_transform(train2['거래점'])

In [118]:
train2

,거래일시,적요,보낸분/받는분,출금액,입금액,잔액,거래점,year,month,day
0,2018-12-31 15:12:10,10,96,"18,900",0,"259,432",0,2018,12,31
1,2018-12-29 18:25:34,10,242,"263,730",0,"278,332",0,2018,12,29
2,2018-12-29 13:32:57,10,12,"3,000",0,"542,062",0,2018,12,29
3,2018-12-29 13:31:16,10,12,"1,500",0,"545,062",0,2018,12,29
4,2018-12-29 13:28:11,10,12,"1,900",0,"546,562",0,2018,12,29
...,...,...,...,...,...,...,...,...,...,...
923,2018-01-04 18:04:11,4,26,"31,623",0,909,2,2018,1,4
924,2018-01-03 22:58:55,9,98,"2,386",0,"30,532",0,2018,1,3
925,2018-01-03 19:26:18,10,13,850,0,"2,918",0,2018,1,3
926,2018-01-03 19:11:06,10,60,"12,000",0,"3,768",0,2018,1,3


# 거래일시 변수 년/월/일 합쳐서 하나의 변수로

In [119]:
X = train2
X

,거래일시,적요,보낸분/받는분,출금액,입금액,잔액,거래점,year,month,day
0,2018-12-31 15:12:10,10,96,"18,900",0,"259,432",0,2018,12,31
1,2018-12-29 18:25:34,10,242,"263,730",0,"278,332",0,2018,12,29
2,2018-12-29 13:32:57,10,12,"3,000",0,"542,062",0,2018,12,29
3,2018-12-29 13:31:16,10,12,"1,500",0,"545,062",0,2018,12,29
4,2018-12-29 13:28:11,10,12,"1,900",0,"546,562",0,2018,12,29
...,...,...,...,...,...,...,...,...,...,...
923,2018-01-04 18:04:11,4,26,"31,623",0,909,2,2018,1,4
924,2018-01-03 22:58:55,9,98,"2,386",0,"30,532",0,2018,1,3
925,2018-01-03 19:26:18,10,13,850,0,"2,918",0,2018,1,3
926,2018-01-03 19:11:06,10,60,"12,000",0,"3,768",0,2018,1,3


In [120]:
X['거래일시'] = X['거래일시'].astype(str).str.replace(pat=r'[^A-Za-z0-9]', repl=r' ', regex=True)
X['잔액'] = X['잔액'].astype(str).str.replace(pat=r'[^A-Za-z0-9]', repl=r' ', regex=True)
X['출금액'] = X['출금액'].astype(str).str.replace(pat=r'[^A-Za-z0-9]', repl=r' ', regex=True)

In [121]:
X

,거래일시,적요,보낸분/받는분,출금액,입금액,잔액,거래점,year,month,day
0,2018 12 31 15 12 10,10,96,18 900,0,259 432,0,2018,12,31
1,2018 12 29 18 25 34,10,242,263 730,0,278 332,0,2018,12,29
2,2018 12 29 13 32 57,10,12,3 000,0,542 062,0,2018,12,29
3,2018 12 29 13 31 16,10,12,1 500,0,545 062,0,2018,12,29
4,2018 12 29 13 28 11,10,12,1 900,0,546 562,0,2018,12,29
...,...,...,...,...,...,...,...,...,...,...
923,2018 01 04 18 04 11,4,26,31 623,0,909,2,2018,1,4
924,2018 01 03 22 58 55,9,98,2 386,0,30 532,0,2018,1,3
925,2018 01 03 19 26 18,10,13,850,0,2 918,0,2018,1,3
926,2018 01 03 19 11 06,10,60,12 000,0,3 768,0,2018,1,3


In [122]:
date=[]
balance =[]
out=[]
inner = []
for i in range(0,len(X['거래일시'])):
    date.append(float((X['거래일시'][i]).replace(' ','')[4:-2])/1000)
    balance.append(int((X['잔액'][i]).replace(' ','')))
    out.append(-int((X['출금액'][i]).replace(' ','')))
    inner.append(int((X['입금액'][i]).replace(',','')))
X['거래일시'] = date
X['잔액'] = balance
X['출금액']= out
X['입금액'] = inner

In [123]:
X

,거래일시,적요,보낸분/받는분,출금액,입금액,잔액,거래점,year,month,day
0,12311.512,10,96,-18900,0,259432,0,2018,12,31
1,12291.825,10,242,-263730,0,278332,0,2018,12,29
2,12291.332,10,12,-3000,0,542062,0,2018,12,29
3,12291.331,10,12,-1500,0,545062,0,2018,12,29
4,12291.328,10,12,-1900,0,546562,0,2018,12,29
...,...,...,...,...,...,...,...,...,...,...
923,1041.804,4,26,-31623,0,909,2,2018,1,4
924,1032.258,9,98,-2386,0,30532,0,2018,1,3
925,1031.926,10,13,-850,0,2918,0,2018,1,3
926,1031.911,10,60,-12000,0,3768,0,2018,1,3


# 데이터 셋 구성

In [47]:
import os, json, glob, sys, numpy as np
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing import sequence
from keras.models import Sequential, Model
from keras.layers import Dense, Embedding, LSTM, Flatten, Dropout, Input, Conv1D, MaxPooling1D, Bidirectional, GlobalMaxPool1D
from keras.utils import np_utils
from keras.callbacks import EarlyStopping, ModelCheckpoint
import keras.backend.tensorflow_backend as K
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import tensorflow as tf
from keras.layers.core import Dense, Activation, Dropout
from keras.layers.recurrent import LSTM
from keras.models import Sequential

In [115]:
Y = X['잔액']
X  = X.drop('잔액',1)

In [71]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2)

# 모델

In [54]:
with K.tf_ops.device('/device:GPU:0'):
    model = Sequential()

    model.add(Dense(units=1024, input_dim=9, activation='relu'))
    model.add(Dense(units=765, activation='relu'))
    model.add(Dense(units=612, activation='relu'))
    model.add(Dense(units=512, activation='relu'))
    model.add(Dense(units=400, activation='relu'))
    model.add(Dense(units=356, activation='relu'))
    model.add(Dense(units=300, activation='relu'))
    model.add(Dense(units=256, activation='relu'))
    model.add(Dense(units=128, activation='relu'))
    model.add(Dense(units=64, activation='relu'))
    model.add(Dense(units=32, activation='relu'))
    model.add(Dense(units=1))

    # 모델 학습과정 설정 
    model.compile(loss='mse', optimizer='adam', metrics=['mae'])

In [55]:
X_train

,거래일시,적요,보낸분/받는분,출금액,입금액,거래점,year,month,day
67,12131.202,10,136,6000,0,0,2018,12,13
43,12192.321,10,99,19000,0,0,2018,12,19
97,12031.201,10,136,12000,0,0,2018,12,3
887,1211.030,10,163,2000,0,0,2018,1,21
70,12121.509,10,65,23500,0,0,2018,12,12
...,...,...,...,...,...,...,...,...,...
394,8241.200,10,167,3000,0,0,2018,8,24
255,10101.439,6,196,20000,0,1,2018,10,10
45,12191.558,10,65,13780,0,0,2018,12,19
17,12251.405,10,132,1300,0,0,2018,12,25


In [56]:
# 학습 조기종료 
from keras.callbacks import EarlyStopping
es = EarlyStopping(patience=10)

# 모델 트레이닝 
hist = model.fit(X_train, y_train, epochs=1000, batch_size=16, callbacks=[es])

Epoch 1/1000
742/742 [==============================] - 0s 532us/step - loss: 34980826953.8329 - mae: 139709.5781
Epoch 2/1000
528/742 [====================>.........] - ETA: 0s - loss: 25718283977.6970 - mae: 117201.6094

C:\Users\munan\Anaconda3\lib\site-packages\keras\callbacks\callbacks.py:846: RuntimeWarning: Early stopping conditioned on metric `val_loss` which is not available. Available metrics are: loss,mae
  (self.monitor, ','.join(list(logs.keys()))), RuntimeWarning


742/742 [==============================] - 0s 317us/step - loss: 28444311011.0189 - mae: 125170.8125
Epoch 3/1000
742/742 [==============================] - 0s 296us/step - loss: 22450456912.7332 - mae: 117032.8594
Epoch 4/1000
742/742 [==============================] - 0s 300us/step - loss: 20700944538.5660 - mae: 110958.2188
Epoch 5/1000
742/742 [==============================] - 0s 305us/step - loss: 18402148434.8032 - mae: 109487.8047
Epoch 6/1000
742/742 [==============================] - 0s 301us/step - loss: 21846667970.5876 - mae: 114197.4688
Epoch 7/1000
742/742 [==============================] - 0s 308us/step - loss: 17771562544.3019 - mae: 107746.7891
Epoch 8/1000
742/742 [==============================] - 0s 298us/step - loss: 18824542500.5714 - mae: 109727.4844
Epoch 9/1000
742/742 [==============================] - 0s 298us/step - loss: 17670476017.5094 - mae: 108228.5703
Epoch 10/1000
742/742 [==============================] - 0s 298us/step - loss: 17362758465.5526 - mae

742/742 [==============================] - 0s 298us/step - loss: 17032008060.8949 - mae: 106282.5000
Epoch 74/1000
742/742 [==============================] - 0s 302us/step - loss: 17219055240.6253 - mae: 106278.2656
Epoch 75/1000
742/742 [==============================] - 0s 301us/step - loss: 16760879540.0970 - mae: 104984.9844
Epoch 76/1000
742/742 [==============================] - 0s 302us/step - loss: 16651809976.9272 - mae: 105599.3516
Epoch 77/1000
742/742 [==============================] - 0s 306us/step - loss: 16921001945.3585 - mae: 106355.7188
Epoch 78/1000
742/742 [==============================] - 0s 304us/step - loss: 16848626925.3693 - mae: 105806.8594
Epoch 79/1000
742/742 [==============================] - 0s 304us/step - loss: 18177494621.8437 - mae: 108162.8672
Epoch 80/1000
742/742 [==============================] - 0s 301us/step - loss: 17078243063.0296 - mae: 106876.4062
Epoch 81/1000
742/742 [==============================] - 0s 297us/step - loss: 16837891497.056

Epoch 144/1000
742/742 [==============================] - 0s 301us/step - loss: 16786644444.1186 - mae: 106351.2031
Epoch 145/1000
742/742 [==============================] - 0s 294us/step - loss: 17023864020.5283 - mae: 104691.6641
Epoch 146/1000
742/742 [==============================] - 0s 305us/step - loss: 16275221633.7251 - mae: 104206.1016
Epoch 147/1000
742/742 [==============================] - 0s 302us/step - loss: 16787165526.2534 - mae: 105101.3672
Epoch 148/1000
742/742 [==============================] - 0s 305us/step - loss: 16259326688.9488 - mae: 104049.4453
Epoch 149/1000
742/742 [==============================] - 0s 306us/step - loss: 16762744302.0593 - mae: 105108.4844
Epoch 150/1000
742/742 [==============================] - 0s 301us/step - loss: 16660951978.4367 - mae: 105180.7734
Epoch 151/1000
742/742 [==============================] - 0s 300us/step - loss: 16327530186.8679 - mae: 104634.8125
Epoch 152/1000
742/742 [==============================] - 0s 300us/step 

...

742/742 [==============================] - 0s 298us/step - loss: 6637365173.4771 - mae: 55136.2695
Epoch 710/1000
742/742 [==============================] - 0s 300us/step - loss: 5864988787.2345 - mae: 50851.5078
Epoch 711/1000
742/742 [==============================] - 0s 302us/step - loss: 7067242880.3450 - mae: 56333.9336
Epoch 712/1000
742/742 [==============================] - 0s 300us/step - loss: 5315572264.8841 - mae: 47516.7656
Epoch 713/1000
742/742 [==============================] - 0s 296us/step - loss: 5674817299.3208 - mae: 48778.9336
Epoch 714/1000
742/742 [==============================] - 0s 302us/step - loss: 6297223926.3396 - mae: 51562.4297
Epoch 715/1000
742/742 [==============================] - 0s 302us/step - loss: 5221922962.1132 - mae: 47519.0898
Epoch 716/1000
742/742 [==============================] - 0s 296us/step - loss: 6431634164.2695 - mae: 50687.1523
Epoch 717/1000
742/742 [==============================] - 0s 300us/step - loss: 7386229920.4313 - mae: 

Epoch 781/1000
742/742 [==============================] - 0s 297us/step - loss: 6220505470.2749 - mae: 49964.7539
Epoch 782/1000
742/742 [==============================] - 0s 297us/step - loss: 5222385030.5553 - mae: 47878.2852
Epoch 783/1000
742/742 [==============================] - 0s 298us/step - loss: 5290924868.4852 - mae: 46274.9180
Epoch 784/1000
742/742 [==============================] - 0s 300us/step - loss: 4455404690.9757 - mae: 42847.5469
Epoch 785/1000
742/742 [==============================] - 0s 294us/step - loss: 6716597472.9488 - mae: 53543.5703
Epoch 786/1000
742/742 [==============================] - 0s 298us/step - loss: 7171027110.9865 - mae: 55557.4609
Epoch 787/1000
742/742 [==============================] - 0s 294us/step - loss: 6102340504.4960 - mae: 49922.1094
Epoch 788/1000
742/742 [==============================] - 0s 298us/step - loss: 4067125496.4097 - mae: 41565.7617
Epoch 789/1000
742/742 [==============================] - 0s 305us/step - loss: 39953195

742/742 [==============================] - 0s 296us/step - loss: 5122530156.7655 - mae: 46141.9336
Epoch 853/1000
742/742 [==============================] - 0s 302us/step - loss: 3828508631.9784 - mae: 39590.3477
Epoch 854/1000
742/742 [==============================] - 0s 297us/step - loss: 5201943845.2615 - mae: 42878.4961
Epoch 855/1000
742/742 [==============================] - 0s 296us/step - loss: 5912207956.5283 - mae: 49164.5703
Epoch 856/1000
742/742 [==============================] - 0s 296us/step - loss: 3344103997.4124 - mae: 35178.3711
Epoch 857/1000
742/742 [==============================] - 0s 301us/step - loss: 4660285939.4070 - mae: 42485.3438
Epoch 858/1000
742/742 [==============================] - 0s 296us/step - loss: 7226684620.2480 - mae: 51289.3594
Epoch 859/1000
742/742 [==============================] - 0s 300us/step - loss: 11016348625.0782 - mae: 71836.5703
Epoch 860/1000
742/742 [==============================] - 0s 297us/step - loss: 5037791800.5822 - mae:

Epoch 924/1000
742/742 [==============================] - 0s 301us/step - loss: 2753524482.7601 - mae: 31637.6367
Epoch 925/1000
742/742 [==============================] - 0s 301us/step - loss: 3655041091.6226 - mae: 38188.9648
Epoch 926/1000
742/742 [==============================] - 0s 301us/step - loss: 3584450626.2426 - mae: 37842.8906
Epoch 927/1000
742/742 [==============================] - 0s 296us/step - loss: 3977009011.9245 - mae: 37699.9336
Epoch 928/1000
742/742 [==============================] - 0s 296us/step - loss: 6313787241.5741 - mae: 51051.5938
Epoch 929/1000
742/742 [==============================] - 0s 294us/step - loss: 10789283383.8922 - mae: 74591.1719
Epoch 930/1000
742/742 [==============================] - 0s 305us/step - loss: 6102405120.6900 - mae: 53612.4219
Epoch 931/1000
742/742 [==============================] - 0s 298us/step - loss: 3215104109.0243 - mae: 37466.6719
Epoch 932/1000
742/742 [==============================] - 0s 297us/step - loss: 7219860

742/742 [==============================] - 0s 293us/step - loss: 1853729016.7547 - mae: 24976.0703
Epoch 996/1000
742/742 [==============================] - 0s 296us/step - loss: 3410281207.3747 - mae: 35078.0586
Epoch 997/1000
742/742 [==============================] - 0s 296us/step - loss: 2886061128.9704 - mae: 33695.4258
Epoch 998/1000
742/742 [==============================] - 0s 305us/step - loss: 3253872846.4906 - mae: 32943.2930
Epoch 999/1000
742/742 [==============================] - 0s 296us/step - loss: 3782463044.3127 - mae: 37247.4141
Epoch 1000/1000
742/742 [==============================] - 0s 293us/step - loss: 2821962010.2210 - mae: 32556.9414


In [58]:
y_pred=model.predict(X_test)
y_pred

array([[348876.2   ],
       [171361.77  ],
       [484336.56  ],
       [368381.3   ],
       [210297.62  ],
       [ 23219.008 ],
       [109020.555 ],
       [275798.62  ],
       [294770.66  ],
       [ 28008.602 ],
       [ 93080.6   ],
       [ 18358.3   ],
       [ 46217.137 ],
       [321357.12  ],
       [353824.2   ],
       [ 30045.938 ],
       [254870.27  ],
       [ 54021.793 ],
       [159011.16  ],
       [329120.56  ],
       [  2973.5095],
       [ 35309.535 ],
       [ 48483.37  ],
       [ 42454.6   ],
       [143494.89  ],
       [153917.48  ],
       [ 75883.8   ],
       [ 21673.215 ],
       [277171.06  ],
       [124243.7   ],
       [ 68688.89  ],
       [441739.47  ],
       [187301.25  ],
       [ 58366.594 ],
       [150896.05  ],
       [ 34235.465 ],
       [ 17127.879 ],
       [ 26822.465 ],
       [531908.    ],
       [307241.44  ],
       [256137.19  ],
       [263875.22  ],
       [167051.1   ],
       [ 80702.43  ],
       [136273.05  ],
       [ 3

# 이대로 쓰면 다음주 잔고 예측

In [160]:
X2 = X[-20:]
X2 = pd.concat([X,X2],ignore_index =True)
X2

,거래일시,적요,보낸분/받는분,출금액,입금액,잔액,거래점,year,month,day
0,12311.512,10,96,-18900,0,259432,0,2018,12,31
1,12291.825,10,242,-263730,0,278332,0,2018,12,29
2,12291.332,10,12,-3000,0,542062,0,2018,12,29
3,12291.331,10,12,-1500,0,545062,0,2018,12,29
4,12291.328,10,12,-1900,0,546562,0,2018,12,29
...,...,...,...,...,...,...,...,...,...,...
943,1041.804,4,26,-31623,0,909,2,2018,1,4
944,1032.258,9,98,-2386,0,30532,0,2018,1,3
945,1031.926,10,13,-850,0,2918,0,2018,1,3
946,1031.911,10,60,-12000,0,3768,0,2018,1,3


In [161]:
Y2 = X2['잔액']
X2  = X2.drop('잔액',1)

In [163]:
X2

,거래일시,적요,보낸분/받는분,출금액,입금액,거래점,year,month,day
0,12311.512,10,96,-18900,0,0,2018,12,31
1,12291.825,10,242,-263730,0,0,2018,12,29
2,12291.332,10,12,-3000,0,0,2018,12,29
3,12291.331,10,12,-1500,0,0,2018,12,29
4,12291.328,10,12,-1900,0,0,2018,12,29
...,...,...,...,...,...,...,...,...,...
943,1041.804,4,26,-31623,0,2,2018,1,4
944,1032.258,9,98,-2386,0,0,2018,1,3
945,1031.926,10,13,-850,0,0,2018,1,3
946,1031.911,10,60,-12000,0,0,2018,1,3


# 안아껴서 쓰면 x2로 통일

In [175]:
x2_train = X2[:-20]
x2_test = X2[-20:]
y2_train = Y2[:-20]
y2_test = Y2[-20:]

In [176]:
# 학습 조기종료 
from keras.callbacks import EarlyStopping
es = EarlyStopping(patience=10)

# 모델 트레이닝 
hist = model.fit(x2_train, y2_train, epochs=1000, batch_size=16, callbacks=[es])

Epoch 1/1000
928/928 [==============================] - 0s 321us/step - loss: 2233877214.3448 - mae: 24836.4453
Epoch 2/1000
528/928 [================>.............] - ETA: 0s - loss: 1916545645.0909 - mae: 22646.9824

C:\Users\munan\Anaconda3\lib\site-packages\keras\callbacks\callbacks.py:846: RuntimeWarning: Early stopping conditioned on metric `val_loss` which is not available. Available metrics are: loss,mae
  (self.monitor, ','.join(list(logs.keys()))), RuntimeWarning


928/928 [==============================] - 0s 319us/step - loss: 2736409359.7241 - mae: 28077.9023
Epoch 3/1000
928/928 [==============================] - 0s 312us/step - loss: 4719510822.6207 - mae: 38806.8633
Epoch 4/1000
928/928 [==============================] - 0s 308us/step - loss: 3545396434.2069 - mae: 31824.0625
Epoch 5/1000
928/928 [==============================] - 0s 314us/step - loss: 1842800229.2414 - mae: 24249.0801
Epoch 6/1000
928/928 [==============================] - 0s 316us/step - loss: 1685949729.3793 - mae: 21466.7031
Epoch 7/1000
928/928 [==============================] - 0s 303us/step - loss: 1023086222.6207 - mae: 17416.2578
Epoch 8/1000
928/928 [==============================] - 0s 308us/step - loss: 877054328.1379 - mae: 15788.5225
Epoch 9/1000
928/928 [==============================] - 0s 297us/step - loss: 2543796382.7586 - mae: 25983.0117
Epoch 10/1000
928/928 [==============================] - 0s 297us/step - loss: 2610347910.6207 - mae: 25989.1797
Epoch

928/928 [==============================] - 0s 290us/step - loss: 1012060600.0000 - mae: 17875.4355
Epoch 75/1000
928/928 [==============================] - 0s 301us/step - loss: 915261950.3448 - mae: 17451.0176
Epoch 76/1000
928/928 [==============================] - 0s 301us/step - loss: 682033543.3103 - mae: 14839.4492
Epoch 77/1000
928/928 [==============================] - 0s 304us/step - loss: 2077227165.1034 - mae: 26932.4355
Epoch 78/1000
928/928 [==============================] - 0s 312us/step - loss: 1277034084.8276 - mae: 19792.9883
Epoch 79/1000
928/928 [==============================] - 0s 303us/step - loss: 670793565.3793 - mae: 14310.7734
Epoch 80/1000
928/928 [==============================] - 0s 294us/step - loss: 2124881714.8966 - mae: 23683.5273
Epoch 81/1000
928/928 [==============================] - 0s 298us/step - loss: 1721154040.5517 - mae: 24708.1758
Epoch 82/1000
928/928 [==============================] - 0s 303us/step - loss: 1078229022.3448 - mae: 17512.4922


928/928 [==============================] - 0s 297us/step - loss: 1868951638.6207 - mae: 23149.7539
Epoch 147/1000
928/928 [==============================] - 0s 303us/step - loss: 1009982462.2069 - mae: 16931.9160
Epoch 148/1000
928/928 [==============================] - 0s 290us/step - loss: 786165811.5862 - mae: 15321.3477
Epoch 149/1000
928/928 [==============================] - 0s 305us/step - loss: 921592325.7241 - mae: 15370.8477
Epoch 150/1000
928/928 [==============================] - 0s 301us/step - loss: 2906017793.9310 - mae: 30402.8828
Epoch 151/1000
928/928 [==============================] - 0s 306us/step - loss: 2046501841.9310 - mae: 23650.6367
Epoch 152/1000
928/928 [==============================] - 0s 308us/step - loss: 1403730489.9310 - mae: 18526.1133
Epoch 153/1000
928/928 [==============================] - 0s 308us/step - loss: 2278177059.5862 - mae: 28397.0391
Epoch 154/1000
928/928 [==============================] - 0s 303us/step - loss: 1304110431.4483 - mae: 20

...

928/928 [==============================] - 0s 297us/step - loss: 863630405.7931 - mae: 14533.6455
Epoch 650/1000
928/928 [==============================] - 0s 291us/step - loss: 664191205.3103 - mae: 13259.6934
Epoch 651/1000
928/928 [==============================] - 0s 292us/step - loss: 1649287203.3103 - mae: 21020.0273
Epoch 652/1000
928/928 [==============================] - 0s 298us/step - loss: 2014986188.2759 - mae: 21322.3047
Epoch 653/1000
928/928 [==============================] - 0s 297us/step - loss: 1555227457.9310 - mae: 19061.8125
Epoch 654/1000
928/928 [==============================] - 0s 288us/step - loss: 1097189835.6552 - mae: 17222.0371
Epoch 655/1000
928/928 [==============================] - 0s 289us/step - loss: 1845087593.1034 - mae: 18009.1230
Epoch 656/1000
928/928 [==============================] - 0s 293us/step - loss: 2424851070.6207 - mae: 25315.2246
Epoch 657/1000
928/928 [==============================] - 0s 291us/step - loss: 7857435448.2759 - mae: 49

928/928 [==============================] - 0s 291us/step - loss: 1188285126.2759 - mae: 17242.1699
Epoch 722/1000
928/928 [==============================] - 0s 292us/step - loss: 6053323937.1034 - mae: 42464.4219
Epoch 723/1000
928/928 [==============================] - 0s 296us/step - loss: 2419231189.5172 - mae: 25637.6641
Epoch 724/1000
928/928 [==============================] - 0s 289us/step - loss: 971058966.2069 - mae: 16635.3652
Epoch 725/1000
928/928 [==============================] - 0s 297us/step - loss: 447825786.4138 - mae: 11328.6494
Epoch 726/1000
928/928 [==============================] - 0s 293us/step - loss: 2671743168.0000 - mae: 27524.1680
Epoch 727/1000
928/928 [==============================] - 0s 290us/step - loss: 2003735676.9655 - mae: 21008.1934
Epoch 728/1000
928/928 [==============================] - 0s 288us/step - loss: 996140830.2759 - mae: 15721.3721
Epoch 729/1000
928/928 [==============================] - 0s 291us/step - loss: 5348795921.1034 - mae: 404

Epoch 793/1000
928/928 [==============================] - 0s 298us/step - loss: 1612246093.2414 - mae: 22085.7324
Epoch 794/1000
928/928 [==============================] - 0s 294us/step - loss: 1167057401.5862 - mae: 16411.5547
Epoch 795/1000
928/928 [==============================] - 0s 294us/step - loss: 650956293.0345 - mae: 12887.0146
Epoch 796/1000
928/928 [==============================] - 0s 290us/step - loss: 412766856.8621 - mae: 10307.7021
Epoch 797/1000
928/928 [==============================] - 0s 291us/step - loss: 433652593.5862 - mae: 11172.0840
Epoch 798/1000
928/928 [==============================] - 0s 291us/step - loss: 634533341.5172 - mae: 12582.9746
Epoch 799/1000
928/928 [==============================] - 0s 290us/step - loss: 453904203.7931 - mae: 9921.4160
Epoch 800/1000
928/928 [==============================] - 0s 297us/step - loss: 1079711907.5172 - mae: 13557.0068
Epoch 801/1000
928/928 [==============================] - 0s 291us/step - loss: 726737305.1034

928/928 [==============================] - 0s 299us/step - loss: 408158766.1207 - mae: 10197.0830
Epoch 866/1000
928/928 [==============================] - 0s 304us/step - loss: 791815994.4483 - mae: 12195.6777
Epoch 867/1000
928/928 [==============================] - 0s 297us/step - loss: 1889124049.6552 - mae: 21347.0039
Epoch 868/1000
928/928 [==============================] - 0s 299us/step - loss: 1175097512.5517 - mae: 17388.0586
Epoch 869/1000
928/928 [==============================] - 0s 299us/step - loss: 326959701.1034 - mae: 8812.6484
Epoch 870/1000
928/928 [==============================] - 0s 296us/step - loss: 564475540.2759 - mae: 11224.1553
Epoch 871/1000
928/928 [==============================] - 0s 305us/step - loss: 757723897.3793 - mae: 12181.0273
Epoch 872/1000
928/928 [==============================] - 0s 308us/step - loss: 665243722.4828 - mae: 10057.3770
Epoch 873/1000
928/928 [==============================] - 0s 307us/step - loss: 2245471742.5862 - mae: 24770.8

928/928 [==============================] - 0s 303us/step - loss: 1144252139.8621 - mae: 15661.4443
Epoch 938/1000
928/928 [==============================] - 0s 300us/step - loss: 767968150.0690 - mae: 13736.4229
Epoch 939/1000
928/928 [==============================] - 0s 303us/step - loss: 476650177.2414 - mae: 10450.7129
Epoch 940/1000
928/928 [==============================] - 0s 298us/step - loss: 420494770.7931 - mae: 10587.8398
Epoch 941/1000
928/928 [==============================] - 0s 293us/step - loss: 1480825316.1724 - mae: 18370.3496
Epoch 942/1000
928/928 [==============================] - 0s 299us/step - loss: 1640113562.4828 - mae: 23761.3477
Epoch 943/1000
928/928 [==============================] - 0s 296us/step - loss: 1390550792.0000 - mae: 18144.5410
Epoch 944/1000
928/928 [==============================] - 0s 303us/step - loss: 3102090464.5517 - mae: 30202.3203
Epoch 945/1000
928/928 [==============================] - 0s 297us/step - loss: 2282165005.3793 - mae: 247

In [177]:
y2_pred=model.predict(x2_test)
y2_pred = list(map(int,y2_pred))
y2_pred

[276431,
 302954,
 138504,
 132755,
 199421,
 232377,
 225819,
 230132,
 239187,
 226987,
 227751,
 252985,
 248789,
 247323,
 1002,
 -202,
 27432,
 2713,
 5020,
 16559]

# 아껴서 쓰면 다음주 잔고 예측

In [155]:
X3 = X[-20:]
X3 = pd.concat([X,X3],ignore_index =True)
X3

,거래일시,적요,보낸분/받는분,출금액,입금액,잔액,거래점,year,month,day
0,12311.512,10,96,-18900,0,259432,0,2018,12,31
1,12291.825,10,242,-263730,0,278332,0,2018,12,29
2,12291.332,10,12,-3000,0,542062,0,2018,12,29
3,12291.331,10,12,-1500,0,545062,0,2018,12,29
4,12291.328,10,12,-1900,0,546562,0,2018,12,29
...,...,...,...,...,...,...,...,...,...,...
943,1041.804,4,26,-31623,0,909,2,2018,1,4
944,1032.258,9,98,-2386,0,30532,0,2018,1,3
945,1031.926,10,13,-850,0,2918,0,2018,1,3
946,1031.911,10,60,-12000,0,3768,0,2018,1,3


In [156]:
a = list(X3['출금액'][-20:])
k=[]
for i in range(len(a)):
    k.append(int(a[i]*0.8))
k = pd.Series(k)

In [157]:
X3['출금액'][-20:] = k

C:\Users\munan\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [158]:
X3

,거래일시,적요,보낸분/받는분,출금액,입금액,잔액,거래점,year,month,day
0,12311.512,10,96,-18900,0,259432,0,2018,12,31
1,12291.825,10,242,-263730,0,278332,0,2018,12,29
2,12291.332,10,12,-3000,0,542062,0,2018,12,29
3,12291.331,10,12,-1500,0,545062,0,2018,12,29
4,12291.328,10,12,-1900,0,546562,0,2018,12,29
...,...,...,...,...,...,...,...,...,...,...
943,1041.804,4,26,-25298,0,909,2,2018,1,4
944,1032.258,9,98,-1908,0,30532,0,2018,1,3
945,1031.926,10,13,-680,0,2918,0,2018,1,3
946,1031.911,10,60,-9600,0,3768,0,2018,1,3


In [159]:
Y3 = X3['잔액']
X3 = X3.drop('잔액',1)

In [164]:
X3

,거래일시,적요,보낸분/받는분,출금액,입금액,거래점,year,month,day
0,12311.512,10,96,-18900,0,0,2018,12,31
1,12291.825,10,242,-263730,0,0,2018,12,29
2,12291.332,10,12,-3000,0,0,2018,12,29
3,12291.331,10,12,-1500,0,0,2018,12,29
4,12291.328,10,12,-1900,0,0,2018,12,29
...,...,...,...,...,...,...,...,...,...
943,1041.804,4,26,-25298,0,2,2018,1,4
944,1032.258,9,98,-1908,0,0,2018,1,3
945,1031.926,10,13,-680,0,0,2018,1,3
946,1031.911,10,60,-9600,0,0,2018,1,3


# 아낄경우 변수명 x3로 통일

In [169]:
x3_train = X3[:-20]
x3_test = X3[-20:]
y3_train = Y3[:-20]
y3_test = Y3[-20:]

In [173]:
x3_test

,거래일시,적요,보낸분/받는분,출금액,입금액,거래점,year,month,day
928,1102.053,6,195,-7200,0,1,2018,1,10
929,1101.916,10,61,-13600,0,0,2018,1,10
930,1101.217,10,228,-1600,0,0,2018,1,10
931,1101.048,10,14,-59040,0,0,2018,1,10
932,1100.959,10,245,-15200,0,0,2018,1,10
933,1091.200,10,229,-2000,0,0,2018,1,9
934,1081.813,10,93,-9600,0,0,2018,1,8
935,1081.210,10,228,-1840,0,0,2018,1,8
936,1081.132,10,214,-2480,0,0,2018,1,8
937,1071.547,10,47,-4840,0,0,2018,1,7


In [170]:
# 학습 조기종료 
from keras.callbacks import EarlyStopping
es = EarlyStopping(patience=10)

# 모델 트레이닝 
hist = model.fit(x3_train, y3_train, epochs=1000, batch_size=16, callbacks=[es])

Epoch 1/1000
928/928 [==============================] - 0s 322us/step - loss: 1076062866043.5862 - mae: 271200.8125
Epoch 2/1000
512/928 [===============>..............] - ETA: 0s - loss: 35792831072.0000 - mae: 138999.2812

C:\Users\munan\Anaconda3\lib\site-packages\keras\callbacks\callbacks.py:846: RuntimeWarning: Early stopping conditioned on metric `val_loss` which is not available. Available metrics are: loss,mae
  (self.monitor, ','.join(list(logs.keys()))), RuntimeWarning


928/928 [==============================] - 0s 318us/step - loss: 35212030252.1379 - mae: 136898.8438
Epoch 3/1000
928/928 [==============================] - 0s 316us/step - loss: 24400090500.4138 - mae: 122274.9531
Epoch 4/1000
928/928 [==============================] - 0s 303us/step - loss: 33567194429.7931 - mae: 127745.2422
Epoch 5/1000
928/928 [==============================] - 0s 318us/step - loss: 23128989890.2069 - mae: 117878.2656
Epoch 6/1000
928/928 [==============================] - 0s 305us/step - loss: 18575570087.7241 - mae: 108826.8984
Epoch 7/1000
928/928 [==============================] - 0s 309us/step - loss: 18137698533.5172 - mae: 108542.2578
Epoch 8/1000
928/928 [==============================] - 0s 304us/step - loss: 19042965133.2414 - mae: 110947.2422
Epoch 9/1000
928/928 [==============================] - 0s 319us/step - loss: 20569869462.0690 - mae: 110839.6094
Epoch 10/1000
928/928 [==============================] - 0s 301us/step - loss: 18434370948.4138 - mae

928/928 [==============================] - 0s 300us/step - loss: 14986977792.0000 - mae: 97690.8125
Epoch 74/1000
928/928 [==============================] - 0s 305us/step - loss: 14633946050.2069 - mae: 96531.0781
Epoch 75/1000
928/928 [==============================] - 0s 291us/step - loss: 14892219904.0000 - mae: 97355.2656
Epoch 76/1000
928/928 [==============================] - 0s 306us/step - loss: 14718316411.5862 - mae: 96067.8984
Epoch 77/1000
928/928 [==============================] - 0s 297us/step - loss: 15153306659.3103 - mae: 98371.1875
Epoch 78/1000
928/928 [==============================] - 0s 296us/step - loss: 15389897931.0345 - mae: 97739.3047
Epoch 79/1000
928/928 [==============================] - 0s 299us/step - loss: 15713840498.7586 - mae: 98939.5078
Epoch 80/1000
928/928 [==============================] - 0s 298us/step - loss: 15285818164.9655 - mae: 97835.7031
Epoch 81/1000
928/928 [==============================] - 0s 291us/step - loss: 14350341234.7586 - mae:

928/928 [==============================] - 0s 300us/step - loss: 11478785147.5862 - mae: 79911.8438
Epoch 145/1000
928/928 [==============================] - 0s 293us/step - loss: 11742251881.9310 - mae: 81594.7812
Epoch 146/1000
928/928 [==============================] - 0s 305us/step - loss: 11174984386.2069 - mae: 78931.4766
Epoch 147/1000
928/928 [==============================] - 0s 306us/step - loss: 11190630934.0690 - mae: 78282.0469
Epoch 148/1000
928/928 [==============================] - 0s 304us/step - loss: 14344941011.8621 - mae: 89262.2812
Epoch 149/1000
928/928 [==============================] - 0s 298us/step - loss: 11175897856.0000 - mae: 77967.3906
Epoch 150/1000
928/928 [==============================] - 0s 297us/step - loss: 11316491550.8966 - mae: 78304.5547
Epoch 151/1000
928/928 [==============================] - 0s 314us/step - loss: 12729565422.3448 - mae: 79852.6172
Epoch 152/1000
928/928 [==============================] - 0s 303us/step - loss: 15128983340.137

...

Epoch 644/1000
928/928 [==============================] - 0s 302us/step - loss: 4256861292.6897 - mae: 38313.9297
Epoch 645/1000
928/928 [==============================] - 0s 322us/step - loss: 6282544285.7931 - mae: 48312.6758
Epoch 646/1000
928/928 [==============================] - 0s 294us/step - loss: 3274012245.5172 - mae: 35315.5312
Epoch 647/1000
928/928 [==============================] - 0s 300us/step - loss: 2735009735.1724 - mae: 31875.4961
Epoch 648/1000
928/928 [==============================] - 0s 292us/step - loss: 3962735557.5172 - mae: 35795.0273
Epoch 649/1000
928/928 [==============================] - 0s 299us/step - loss: 7911437935.4483 - mae: 56222.6953
Epoch 650/1000
928/928 [==============================] - 0s 317us/step - loss: 4786564339.8621 - mae: 43161.6250
Epoch 651/1000
928/928 [==============================] - 0s 306us/step - loss: 1844762253.2414 - mae: 26328.9316
Epoch 652/1000
928/928 [==============================] - 0s 301us/step - loss: 24994426

928/928 [==============================] - 0s 306us/step - loss: 2958965377.9310 - mae: 31096.3750
Epoch 716/1000
928/928 [==============================] - 0s 307us/step - loss: 2024176132.4138 - mae: 27193.0918
Epoch 717/1000
928/928 [==============================] - 0s 313us/step - loss: 1876880027.0345 - mae: 25720.5703
Epoch 718/1000
928/928 [==============================] - 0s 306us/step - loss: 2369009175.1724 - mae: 29476.9766
Epoch 719/1000
928/928 [==============================] - 0s 308us/step - loss: 1604575151.4483 - mae: 24062.1406
Epoch 720/1000
928/928 [==============================] - 0s 304us/step - loss: 4616835405.7931 - mae: 39818.2734
Epoch 721/1000
928/928 [==============================] - 0s 300us/step - loss: 5352258475.5862 - mae: 43444.6562
Epoch 722/1000
928/928 [==============================] - 0s 303us/step - loss: 2698331931.0345 - mae: 32491.6113
Epoch 723/1000
928/928 [==============================] - 0s 301us/step - loss: 5730714552.2759 - mae: 

928/928 [==============================] - 0s 305us/step - loss: 2771987540.9655 - mae: 30869.5234
Epoch 787/1000
928/928 [==============================] - 0s 301us/step - loss: 2639806118.8966 - mae: 30684.5273
Epoch 788/1000
928/928 [==============================] - 0s 300us/step - loss: 2852976029.7931 - mae: 30923.7520
Epoch 789/1000
928/928 [==============================] - 0s 306us/step - loss: 4871052469.2414 - mae: 39343.5586
Epoch 790/1000
928/928 [==============================] - 0s 298us/step - loss: 3941443058.2069 - mae: 34845.6289
Epoch 791/1000
928/928 [==============================] - 0s 299us/step - loss: 3720696304.0000 - mae: 36136.4609
Epoch 792/1000
928/928 [==============================] - 0s 310us/step - loss: 3368528195.3103 - mae: 35357.7852
Epoch 793/1000
928/928 [==============================] - 0s 307us/step - loss: 2228854210.2069 - mae: 25445.6797
Epoch 794/1000
928/928 [==============================] - 0s 305us/step - loss: 3636141827.3103 - mae: 

Epoch 858/1000
928/928 [==============================] - 0s 303us/step - loss: 1317698017.3793 - mae: 20661.1836
Epoch 859/1000
928/928 [==============================] - 0s 297us/step - loss: 1599172213.2414 - mae: 21675.0977
Epoch 860/1000
928/928 [==============================] - 0s 307us/step - loss: 1554308591.4483 - mae: 23034.3789
Epoch 861/1000
928/928 [==============================] - 0s 300us/step - loss: 2913624988.1379 - mae: 32517.8066
Epoch 862/1000
928/928 [==============================] - 0s 302us/step - loss: 3289230397.7931 - mae: 31133.7637
Epoch 863/1000
928/928 [==============================] - 0s 297us/step - loss: 3135583800.0000 - mae: 29744.9727
Epoch 864/1000
928/928 [==============================] - 0s 298us/step - loss: 3795115444.6897 - mae: 33561.6328
Epoch 865/1000
928/928 [==============================] - 0s 299us/step - loss: 4370186510.3448 - mae: 41051.2383
Epoch 866/1000
928/928 [==============================] - 0s 303us/step - loss: 32780949

928/928 [==============================] - 0s 300us/step - loss: 3239091525.5172 - mae: 34625.1211
Epoch 930/1000
928/928 [==============================] - 0s 299us/step - loss: 1648723847.7241 - mae: 24250.7422
Epoch 931/1000
928/928 [==============================] - 0s 305us/step - loss: 1311256433.3793 - mae: 20145.6641
Epoch 932/1000
928/928 [==============================] - 0s 296us/step - loss: 961881980.2759 - mae: 18254.0273
Epoch 933/1000
928/928 [==============================] - 0s 303us/step - loss: 1278570731.5862 - mae: 18456.9043
Epoch 934/1000
928/928 [==============================] - 0s 300us/step - loss: 1394106441.1034 - mae: 21497.8086
Epoch 935/1000
928/928 [==============================] - 0s 293us/step - loss: 969925001.4483 - mae: 17210.5781
Epoch 936/1000
928/928 [==============================] - 0s 294us/step - loss: 1371521275.8621 - mae: 20996.7051
Epoch 937/1000
928/928 [==============================] - 0s 302us/step - loss: 851523500.0000 - mae: 168

In [174]:
y3_pred=model.predict(x3_test)
y3_pred = list(map(int,y3_pred))
y3_pred

[256057,
 242669,
 148807,
 239062,
 271285,
 189512,
 300679,
 210131,
 246037,
 236910,
 139509,
 253471,
 255090,
 263838,
 7096,
 24539,
 33137,
 1689,
 4653,
 6309]

In [178]:
train

,거래일시,적요,보낸분/받는분,출금액,입금액,잔액,거래점,year,month,day
0,2018-12-31 15:12:10,체크카드,루틴,"18,900",0,"259,432",KB카드,2018,12,31
1,2018-12-29 18:25:34,체크카드,충북대학교병원,"263,730",0,"278,332",KB카드,2018,12,29
2,2018-12-29 13:32:57,체크카드,(주)케이알산업입장(,"3,000",0,"542,062",KB카드,2018,12,29
3,2018-12-29 13:31:16,체크카드,(주)케이알산업입장(,"1,500",0,"545,062",KB카드,2018,12,29
4,2018-12-29 13:28:11,체크카드,(주)케이알산업입장(,"1,900",0,"546,562",KB카드,2018,12,29
...,...,...,...,...,...,...,...,...,...,...
923,2018-01-04 18:04:11,국민카드,KB카드출금,"31,623",0,909,개인고,2018,1,4
924,2018-01-03 22:58:55,직불출금,마스타해외승인출금,"2,386",0,"30,532",KB카드,2018,1,3
925,2018-01-03 19:26:18,체크카드,(주)코리아세븐청주사,850,0,"2,918",KB카드,2018,1,3
926,2018-01-03 19:11:06,체크카드,까구샵,"12,000",0,"3,768",KB카드,2018,1,3


In [181]:
result = train[-20:]
result['이대로사용시다음주잔고'] = y2_pred
result['20%아낄때다음주잔고'] = y3_pred

C:\Users\munan\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\munan\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [182]:
result

,거래일시,적요,보낸분/받는분,출금액,입금액,잔액,거래점,year,month,day,이대로사용시다음주잔고,20%아낄때다음주잔고
908,2018-01-10 20:53:56,리브머니,윤기경,"9,000",0,"288,243",가경남,2018,1,10,276431,256057
909,2018-01-10 19:16:27,체크카드,꼬부리,"17,000",0,"297,243",KB카드,2018,1,10,302954,242669
910,2018-01-10 12:17:28,체크카드,지에스25청주어울림,"2,000",0,"137,243",KB카드,2018,1,10,138504,148807
911,2018-01-10 10:48:26,체크카드,(주)티몬_스마트로,"73,800",0,"139,243",KB카드,2018,1,10,132755,239062
912,2018-01-10 09:59:24,체크카드,카카오선물하기_카카,"19,000",0,"213,043",KB카드,2018,1,10,199421,271285
913,2018-01-09 12:00:54,체크카드,지에스25청주지웰점,"2,500",0,"232,043",KB카드,2018,1,9,232377,189512
914,2018-01-08 18:13:34,체크카드,롯데시네마청주시네,"12,000",0,"234,543",KB카드,2018,1,8,225819,300679
915,2018-01-08 12:10:16,체크카드,지에스25청주어울림,"2,300",0,"246,319",KB카드,2018,1,8,230132,210131
916,2018-01-08 11:32:17,체크카드,주식회사비지에프포,"3,100",0,"248,619",KB카드,2018,1,8,239187,246037
917,2018-01-07 15:47:49,체크카드,구글플레이,"6,050",0,"250,279",KB카드,2018,1,7,226987,236910


In [186]:
result.to_csv('./result.csv',encoding = 'CP949')